In [2]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #transforms categorical data into numerical format
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime,timedelta

In [3]:
API_KEY='973ed7749164da8ee67a51bcd8b4728b'
BASE_URL='https://api.openweathermap.org/data/2.5/'

In [4]:
def get_current_weather(city):
    url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" 
    response=requests.get(url)
    data=response.json()
    return {
        'city':data['name'],
        'current_temp':round(data['main']['temp']),
        'feels_like':round(data['main']['feels_like']),
        'temp_min':round(data['main']['temp_min']),
        'temp_max':round(data['main']['temp_max']),
        'humidity':round(data['main']['humidity']),
        'description':data['weather'][0]['description'],
        'country':data['sys']['country'],
    }

In [5]:
def read_historical_data(filename):
    df=pd.read_csv(filename)
    df=df.dropna()
    df=df.drop_duplicates()
    return df

In [7]:
def prepare_data(data):
    le=LabelEncoder()
    data['WindGustDir']=le.fit_transform(data['WindGustDir']) #converts catogorical windgustdir data into numerica values
    data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])
    X=data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
    y=data['RainTomorrow']
    return X,y,le

In [ ]:
def train_rain_model(X,y):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) #random_state=42 fixes the randomness, so every time you run the code, you get the same split.
    model=RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Mean squared error for test set")
    print(mean_squared_error(y_test,y_pred))
    return model